# **Final Project**

In [ ]:
'''Imports'''
from src.preprocessing import preprocess

## **Preprocessing and Augmentation**

In [ ]:
'''Calls Preprocessing functions'''
# Set the batch size according to your system's capabilities
BATCH_SIZE = 8

# Call the preprocess function
stack = preprocess(BATCH_SIZE)

## **Training**

In [ ]:
'''Calls training functions'''

## **Prediction**

In [ ]:
'''Calls prediction fucntions in prediction.py and methods from result'''

## **Validation**

In [ ]:
'''Calls the validation fuctions in validation.py and methods from result'''